# 1 Introducción

En el siguiente ejercicio se realiza el intercambio de los elementos de un vector a otro de igual dimensión, cada elemento se reubica en la misma posición en la que se encontraba.

El algoritmo está basado en la función Swap [3] de la biblioteca BLAS [2].
Se busca comparar y analizar la performance del procesamiento de estructuras de una dimensión en GPU y en CPU.

Para su implementación en GPU se desarrolló una función kernel que recibe tres parámetros: La cantidad de elementos de los vectores, el puntero al vector X y el puntero al vector Y. Gracias a ésta, el algoritmo utiliza hilos para realizar los intercambios de una manera más eficiente que si se fuera implementado en CPU.

Para su implementación en CPU no se necesitó más que desarrollar un simple for con el respectivo intercambio de elementos de los vectores. A diferencia de su implementación en GPU, no son necesarias ejecuciones previas del armado del ambiente.


---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo
Ejecución del algoritmo en GPU.

In [ ]:
#@title ## 3.1 Parámetros de ejecución
#@markdown ---
#@markdown ### Especifique la cantidad de elementos de los vectores:
cantidad_N =   10000#@param {type: "number"}
#@markdown ---

#Valida parámetro
if cantidad_N <= 0:
  print('La cantidad de elemtentos debe ser mayor a cero.')
else:
  from datetime import datetime
  from pycuda.compiler import SourceModule
  import pycuda.driver as cuda
  import pycuda.autoinit
  import numpy

  #Inicia tiempo total de ejecución
  tiempo_total = datetime.now()

  #Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  try:
    #Define la memoria de los vectores en CPU y los setea con números
    #enteros aleatorios entre el 1 y el 9
    x_cpu = numpy.random.randint(1,10, cantidad_N)
    x_cpu = x_cpu.astype(numpy.int64())
    y_cpu = numpy.random.randint(1,10, cantidad_N)
    y_cpu = y_cpu.astype(numpy.int64())
  except:
    print('Error al definir en memoria los vectores de CPU.')

  #Muestra los vectores iniciales X e Y
  print("Vector X inicial:", x_cpu)
  print("Vector Y inicial:", y_cpu)

  try:
    #Reservo la memoria GPU.
    x_gpu = cuda.mem_alloc( x_cpu.nbytes )
    y_gpu = cuda.mem_alloc( y_cpu.nbytes )
  except:
    print('Error al reservar la memoria GPU.')

  try:
    #Copia la memoria al GPU.
    cuda.memcpy_htod( x_gpu, x_cpu )
    cuda.memcpy_htod( y_gpu, y_cpu )
  except:
    print('Error al copiar la memoria al GPU.')

  #Define la función kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel_swap( int n, int *X, int *Y )
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    int aux;
    if( idx < n )
    {
      aux = X[idx];
      X[idx] = Y[idx];
      Y[idx] = aux;
    }
  }
  """)

  #Genera la función kernel
  kernel = module.get_function("kernel_swap")

  #Define dimensiones de hilos y bloques
  dim_hilo = 256
  dim_bloque = numpy.int((cantidad_N+dim_hilo-1)/dim_hilo)

  #Inicia el tiempo de GPU
  tiempo_gpu = datetime.now()

  #Llama a la función kernel
  kernel(numpy.int64(cantidad_N), x_gpu, y_gpu, block=(dim_hilo, 1, 1),grid=(dim_bloque, 1,1))

  #Finaliza el tiempo de GPU
  tiempo_gpu = datetime.now() - tiempo_gpu

  try:
    #Copia el resultado desde la memoria GPU a CPU
    cuda.memcpy_dtoh(y_cpu, y_gpu)
    cuda.memcpy_dtoh(x_cpu, x_gpu)
  except:
    print('Error al copiar el resultado desde la memoria GPU a CPU.')

  #Calcula tiempo total de ejecución
  tiempo_total = datetime.now() - tiempo_total

  #Muestra los vectores finales X e Y
  print("Vector X final:", x_cpu)
  print("Vector Y final:", y_cpu)

  #Muestra cantidad de elementos
  print("Cantidad de elementos: ", cantidad_N)

  #Muestra dimensiones de hilos y bloques
  print("Thread x: ", dim_hilo, ", Bloque x:", dim_bloque)

  #Muestra los tiempos de GPU y total de ejecución
  print("Tiempo TOTAL: ", tiempo_en_ms(tiempo_total), "[ms]")
  print("Tiempo GPU  : ", tiempo_en_ms( tiempo_gpu), "[ms]")


---
# 4 Tabla de pasos de ejecución del programa


Procesador  |Función               |Detalle
------------|----------------------|-----------------------------------------
CPU         |pip install pycuda    |Instala en el cuaderno los driver de CUDA para Python.
CPU         |@param                |Lectura del tamaño de vectores desde Colab.
CPU         |if                    |Valida parámetro.
CPU         |import                |Importa los módulos para funcionar.
CPU         |datetime.now()        |Toma el tiempo actual.
CPU         |numpy.random.randint(..)|Inicializa los vectores X e Y con enteros aleatorios entre 1 y 9.
CPU         |astype(..)            |Castea los vectores a int64.
CPU         |print(..)             |Muestra los vectores iniciales X e Y.
**GPU**     |cuda.mem_alloc(..)    |Reserva la memoria en GPU.
**GPU**     |cuda.memcpy_htod(..)  |Copia las memorias desde el CPU al GPU.
CPU         |SourceModule(..)      |Define el código del kernel.
CPU         |module.get_function(..)|Genera la función del kernel GPU
CPU         |dim_tx/dim_bx         |Calcula las dimensiones.
CPU         |datetime.now()        |Toma el tiempo actual para calcular el tiempo de GPU.
**GPU**     |kernel(..)            |Ejecuta el kernel en GPU.
CPU         |datetime.now()        |Toma el tiempo actual para calcular el tiempo de GPU.
CPU         |cuda.memcpy_dtoh(..)  |Copia el resultado desde GPU memoria X a CPU memoria X y GPU memoria X a CPU memoria Y.
CPU         |datetime.now()        |Toma el tiempo actual para calcular el tiempo total de ejecución.
CPU         |print(..)             |Muestra los vectores finales X e Y.
CPU         |print(..)             |Muestra la cantidad de elementos.
CPU         |print(..)             |Muestra dimensiones de hilos y bloques.
CPU         |print(..)             |Informa el total de ejecución.
CPU         |print(..)             |Informa el tiempo de GPU.



---
# 5 Conclusiones

Como una conclusión general y empírica se puede observar que los datos, tras cierta cantidad de ejecuciones, arrojan que el procesamiento a partir de la GPU es muchísimo más eficiente. 

Por ejemplo: 

Ejecución de intercambio de vectores (Con 10000 elementos) en CPU: 11.987 [ms].

Ejecución de intercambio de vectores (Con 10000 elementos) en GPU: 0.163 [ms].

Se puede ver claramente la diferencia de tiempos entre cada ejecución al momento de realizar los intercambios.

Se puede apreciar que si el vector es de un solo elemento no tiene sentido procesarlo con GPU. 

Ejecución de intercambio de vectores (Con 1 elemento) en CPU: 0.089 [ms].

Ejecución de intercambio de vectores (Con 1 elemento) en GPU: 0.109 [ms].

Esto se debe a que como el hilo hace el context switch manejando su estructura de control, se pierde más tiempo que trabajar el intercambio de manera secuencial.





---
# 6 Bibliografía

[1] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[2] Función Swap de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-swap.html)

[3] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[4] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[5] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)

[6] ¿QUÉ ES LA COMPUTACIÓN ACELERADA POR GPU?: [Página Nvidia](https://www.nvidia.com/es-la/drivers/what-is-gpu-computing/)

[7] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)





